In [1]:
import copy
import numpy as np
from sklearn import preprocessing
import h5py
import random
from sklearn.metrics import f1_score, accuracy_score
import math

In [2]:
class Layer:
    # constructor
    def __init__(self, neurons=0, theta=[], b=[], z=[], a=[],delta=[],DELTA_THETA=[],DELTA_BIAS=[],dTheta=[],dBias=[]):
        self.neurons = 0 # neurons count in layer
        self.theta = [] # Weight vector(W)
        self.b=[] # bias
        self.z = [] # hypothesis z = W.T * X + b = here = theta.T * X + b
        self.a = [] # activation function a=sigmoid(z) or relu(z) or anyother(z)
        self.delta = [] # Loss or Error function delta= delta_cross_entropy() or anyother()
        self.DELTA_THETA = [] # only derivative weight vector =dw
        self.DELTA_BIAS = [] # only derivative bias vector =db
        self.dTheta = [] # complete derivation term = (1/m)*(DELTA + (lambda*theta))
        self.dBias = [] # complete derivation term 

    def setNeurons(self, neurons):
        self.neurons = neurons

    def getNeurons(self):
        return self.neurons

    def setTheta(self, theta):
        self.theta = theta

    def getTheta(self):
        return self.theta

    def setB(self, b):
        self.b = b

    def getB(self):
        return self.b

    def setZ(self, z):
        self.z = z

    def getZ(self):
        return self.z

    def setA(self, a):
        self.a = a

    def getA(self):
        return self.a

    def setDelta(self, delta):
        self.delta = delta

    def getDelta(self):
        return self.delta

    def setDELTA_THETA(self, DELTA_THETA):
        self.DELTA_THETA = DELTA_THETA

    def getDELTA_THETA(self):
        return self.DELTA_THETA

    def setDELTA_BIAS(self, DELTA_BIAS):
        self.DELTA_BIAS = DELTA_BIAS

    def getDELTA_BIAS(self):
        return self.DELTA_BIAS
    
    def setDTheta(self, dTheta):
        self.dTheta = dTheta

    def getDTheta(self):
        return self.dTheta
    
    def setDBias(self, dBias):
        self.dBias = dBias

    def getDBias(self):
        return self.dBias
    
    

In [3]:
dataset = h5py.File('../MNIST_Subset-2.h5', 'r')

print(list(dataset.keys()))

['X', 'Y']


In [4]:
data_X = dataset['X']
data_Y = dataset['Y']

X = np.array(data_X.value)
Y = np.array(data_Y.value)

print(X.shape,Y.shape)

# Y=Y.reshape(14251,1)
print(X.shape,Y.shape)

(14251, 28, 28) (14251,)
(14251, 28, 28) (14251,)


In [5]:
print(X.shape)
X=X.reshape(14251,28*28)
print(X.shape)

X = preprocessing.scale(X)

m=X.shape[0]
print('no.of samples:',m)

(14251, 28, 28)
(14251, 784)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


no.of samples: 14251


In [6]:
X=X.T # transposed X now shape=784 x 14251 => now each column is one datapoint
print(X.shape)

(784, 14251)


In [7]:
def sigmoid(z):
    return (1.0/(1.0+np.exp(-z)))

In [8]:
def softmax(z):
#     return np.exp(z)/np.sum(np.exp(z),axis=0)
    return np.nan_to_num(np.divide(np.exp(z),np.sum(np.exp(z),axis=0)))

In [9]:
def stableSoftmax(z):
    exps = np.exp(z - np.max(z))
    return np.divide(exps , np.sum(exps,axis=0))

In [10]:
def delta_cross_entropy(z,y):
    grad = (z-y)/m
    return grad

In [11]:
def crossEntropy(a,y):
    return (-y*np.log(a))/m

In [12]:
def accuracy(y_predicted):
    y_multilabel = []
    for p in y_predicted:
        y_multilabel.append(list(p).index(max(p)))
        
    print(accuracy_score(y_multilabel, Y))

In [13]:
def relu(d):
    ans = np.maximum(d,0)
    return ans

In [14]:
y_actual=[] # changed to 1 at their index

for i in range(Y.shape[0]):
    temp = [0]*10
    index = int(Y[i])
    temp[index] = 1
    y_actual.append(temp)
y_actual=np.array(y_actual).T


In [15]:
l=5
neurons=[784,100,50,50,10]

# l=3
# neurons=[784,100,10]

layers=[]
for i in range(len(neurons)):
    lay=Layer()
    
    if(i!=len(neurons)-1):
        DELTA_THETA=np.zeros((neurons[i+1],neurons[i]))
        theta=np.random.uniform(low=0.1,high=1,size=(neurons[i],neurons[i+1]))/math.sqrt(neurons[i])
        #DELTA_BIAS=
        #bias=
        
        lay.setDELTA_THETA(DELTA_THETA)
        lay.setTheta(theta)
    layers.append(lay)

layers[0].setA(X)
regParam=1
alpha=0.01
maxIterations=20000

for iter in range(maxIterations):
    
    # Forward propagation
    for i in range(1,l):
        z=np.dot(layers[i-1].getTheta().T, layers[i-1].getA())
        if(i==l-1):
#             a=stableSoftmax(z)
            a=softmax(z)
        else:
            a=relu(z)
        layers[i].setZ(z)
        layers[i].setA(a)
    
    # Backward Propagation
    for i in range(l-1,-1,-1):
        loss=None
        if(i==l-1):
            loss=delta_cross_entropy(layers[i].getA(),y_actual)
        else:
            deriv=(layers[i].getA() > 0).astype(int)
#             layers[i].getA()[layers[i].getA() <= 0] =0
#             layers[i].getA()[layers[i].getA() >0] =1
#             deriv=(layers[i].getA() > 0).astype(int)
#             deriv=layers[i].getA()
            
            loss=np.dot(layers[i].getTheta(),layers[i+1].getDelta()) * (deriv)
        
        layers[i].setDelta(loss)
    
    for i in range(0,l-1):
        D=layers[i].getDELTA_THETA() + np.dot(layers[i+1].getDelta(),layers[i].getA().T)
        layers[i].setDELTA_THETA(D)
    
    for i in range(0,l-1):
        dT=(1/m)*(layers[i].getDELTA_THETA().T+(regParam*layers[i].getTheta()))
        layers[i].setDTheta(dT)
    
    print('Iteration:',iter,'--> ',end='')
    accuracy(layers[-1].getA().T)
    if(accuracy(layers[-1].getA().T) == np.nan):
        break
    
    for i in range(0,l-1):
        newTh=layers[i].getTheta()-(alpha*layers[i].getDTheta())
        layers[i].setTheta(newTh)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration: 0 --> 0.0
0.0
Iteration: 1 --> 0.0
0.0
Iteration: 2 --> 0.0
0.0
Iteration: 3 --> 0.0
0.0
Iteration: 4 --> 0.0
0.0
Iteration: 5 --> 0.0
0.0
Iteration: 6 --> 0.0
0.0
Iteration: 7 --> 0.0
0.0
Iteration: 8 --> 0.0
0.0
Iteration: 9 --> 0.0
0.0
Iteration: 10 --> 0.0
0.0
Iteration: 11 --> 0.0
0.0
Iteration: 12 --> 0.0
0.0
Iteration: 13 --> 0.0
0.0
Iteration: 14 --> 0.0
0.0
Iteration: 15 --> 0.0
0.0
Iteration: 16 --> 0.0
0.0
Iteration: 17 --> 0.0
0.0
Iteration: 18 --> 0.0
0.0
Iteration: 19 --> 0.0
0.0
Iteration: 20 --> 0.0
0.0


/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Iteration: 21 --> 0.0
0.0
Iteration: 22 --> 0.0
0.0
Iteration: 23 --> 0.0
0.0
Iteration: 24 --> 0.0
0.0
Iteration: 25 --> 0.0
0.0
Iteration: 26 --> 0.0
0.0
Iteration: 27 --> 0.0
0.0
Iteration: 28 --> 0.0
0.0
Iteration: 29 --> 0.0
0.0
Iteration: 30 --> 0.0
0.0
Iteration: 31 --> 0.0
0.0
Iteration: 32 --> 0.0
0.0
Iteration: 33 --> 0.0016840923443968844
0.0016840923443968844
Iteration: 34 --> 0.25149112342993474
0.25149112342993474
Iteration: 35 --> 0.25338572731738124
0.25338572731738124
Iteration: 36 --> 0.25338572731738124
0.25338572731738124
Iteration: 37 --> 0.25331555680303136
0.25331555680303136
Iteration: 38 --> 0.25331555680303136
0.25331555680303136
Iteration: 39 --> 0.2532453862886815
0.2532453862886815
Iteration: 40 --> 0.2532453862886815
0.2532453862886815
Iteration: 41 --> 0.2532453862886815
0.2532453862886815
Iteration: 42 --> 0.2532453862886815
0.2532453862886815
Iteration: 43 --> 0.2532453862886815
0.2532453862886815
Iteration: 44 --> 0.2532453862886815
0.2532453862886815


Iteration: 170 --> 0.24889481439898956
0.24889481439898956
Iteration: 171 --> 0.2487544733702898
0.2487544733702898
Iteration: 172 --> 0.2485439618272402
0.2485439618272402
Iteration: 173 --> 0.24847379131289032
0.24847379131289032
Iteration: 174 --> 0.24833345028419057
0.24833345028419057
Iteration: 175 --> 0.24833345028419057
0.24833345028419057
Iteration: 176 --> 0.24819310925549085
0.24819310925549085
Iteration: 177 --> 0.2480527682267911
0.2480527682267911
Iteration: 178 --> 0.24791242719809137
0.24791242719809137
Iteration: 179 --> 0.2478422566837415
0.2478422566837415
Iteration: 180 --> 0.24770191565504174
0.24770191565504174
Iteration: 181 --> 0.247561574626342
0.247561574626342
Iteration: 182 --> 0.24749140411199214
0.24749140411199214
Iteration: 183 --> 0.2473510630832924
0.2473510630832924
Iteration: 184 --> 0.24728089256894253
0.24728089256894253
Iteration: 185 --> 0.24721072205459266
0.24721072205459266
Iteration: 186 --> 0.24721072205459266
0.24721072205459266
Iteration: 

...

Iteration: 1205 --> 0.0
0.0
Iteration: 1206 --> 0.0
0.0
Iteration: 1207 --> 0.0
0.0
Iteration: 1208 --> 0.0
0.0
Iteration: 1209 --> 0.0
0.0
Iteration: 1210 --> 0.0
0.0
Iteration: 1211 --> 0.0
0.0
Iteration: 1212 --> 0.0
0.0
Iteration: 1213 --> 0.0
0.0
Iteration: 1214 --> 0.0
0.0
Iteration: 1215 --> 0.0
0.0
Iteration: 1216 --> 0.0
0.0
Iteration: 1217 --> 0.0
0.0
Iteration: 1218 --> 0.0
0.0
Iteration: 1219 --> 0.0
0.0
Iteration: 1220 --> 0.0
0.0
Iteration: 1221 --> 0.0
0.0
Iteration: 1222 --> 0.0
0.0
Iteration: 1223 --> 0.0
0.0
Iteration: 1224 --> 0.0
0.0
Iteration: 1225 --> 0.0
0.0
Iteration: 1226 --> 0.0
0.0
Iteration: 1227 --> 0.0
0.0
Iteration: 1228 --> 0.0
0.0
Iteration: 1229 --> 0.0
0.0
Iteration: 1230 --> 0.0
0.0
Iteration: 1231 --> 0.0
0.0
Iteration: 1232 --> 0.0
0.0
Iteration: 1233 --> 0.0
0.0
Iteration: 1234 --> 0.0
0.0
Iteration: 1235 --> 0.0
0.0
Iteration: 1236 --> 0.0
0.0
Iteration: 1237 --> 0.0
0.0
Iteration: 1238 --> 0.0
0.0
Iteration: 1239 --> 0.0
0.0
Iteration: 1240 --> 

Iteration: 1498 --> 0.0
0.0
Iteration: 1499 --> 0.0
0.0
Iteration: 1500 --> 0.0
0.0
Iteration: 1501 --> 0.0
0.0
Iteration: 1502 --> 0.0
0.0
Iteration: 1503 --> 0.0
0.0
Iteration: 1504 --> 0.0
0.0
Iteration: 1505 --> 0.0
0.0
Iteration: 1506 --> 0.0
0.0
Iteration: 1507 --> 0.0
0.0
Iteration: 1508 --> 0.0
0.0
Iteration: 1509 --> 0.0
0.0
Iteration: 1510 --> 0.0
0.0
Iteration: 1511 --> 0.0
0.0
Iteration: 1512 --> 0.0
0.0
Iteration: 1513 --> 0.0
0.0
Iteration: 1514 --> 0.0
0.0
Iteration: 1515 --> 0.0
0.0
Iteration: 1516 --> 0.0
0.0
Iteration: 1517 --> 0.0
0.0
Iteration: 1518 --> 0.0
0.0
Iteration: 1519 --> 0.0
0.0
Iteration: 1520 --> 0.0
0.0
Iteration: 1521 --> 0.0
0.0
Iteration: 1522 --> 0.0
0.0
Iteration: 1523 --> 0.0
0.0
Iteration: 1524 --> 0.0
0.0
Iteration: 1525 --> 0.0
0.0
Iteration: 1526 --> 0.0
0.0
Iteration: 1527 --> 0.0
0.0
Iteration: 1528 --> 0.0
0.0
Iteration: 1529 --> 0.0
0.0
Iteration: 1530 --> 0.0
0.0
Iteration: 1531 --> 0.0
0.0
Iteration: 1532 --> 0.0
0.0
Iteration: 1533 --> 

Iteration: 1791 --> 0.0
0.0
Iteration: 1792 --> 0.0
0.0
Iteration: 1793 --> 0.0
0.0
Iteration: 1794 --> 0.0
0.0
Iteration: 1795 --> 0.0
0.0
Iteration: 1796 --> 0.0
0.0
Iteration: 1797 --> 0.0
0.0
Iteration: 1798 --> 0.0
0.0
Iteration: 1799 --> 0.0
0.0
Iteration: 1800 --> 0.0
0.0
Iteration: 1801 --> 0.0
0.0
Iteration: 1802 --> 0.0
0.0
Iteration: 1803 --> 0.0
0.0
Iteration: 1804 --> 0.0
0.0
Iteration: 1805 --> 0.0
0.0
Iteration: 1806 --> 0.0
0.0
Iteration: 1807 --> 0.0
0.0
Iteration: 1808 --> 0.0
0.0
Iteration: 1809 --> 0.0
0.0
Iteration: 1810 --> 0.0
0.0
Iteration: 1811 --> 0.0
0.0
Iteration: 1812 --> 0.0
0.0
Iteration: 1813 --> 0.0
0.0
Iteration: 1814 --> 0.0
0.0
Iteration: 1815 --> 0.0
0.0
Iteration: 1816 --> 0.0
0.0
Iteration: 1817 --> 0.0
0.0
Iteration: 1818 --> 0.0
0.0
Iteration: 1819 --> 0.0
0.0
Iteration: 1820 --> 0.0
0.0
Iteration: 1821 --> 0.0
0.0
Iteration: 1822 --> 0.0
0.0
Iteration: 1823 --> 0.0
0.0
Iteration: 1824 --> 0.0
0.0
Iteration: 1825 --> 0.0
0.0
Iteration: 1826 --> 

Iteration: 2084 --> 0.0
0.0
Iteration: 2085 --> 0.0
0.0
Iteration: 2086 --> 0.0
0.0
Iteration: 2087 --> 0.0
0.0
Iteration: 2088 --> 0.0
0.0
Iteration: 2089 --> 0.0
0.0
Iteration: 2090 --> 0.0
0.0
Iteration: 2091 --> 0.0
0.0
Iteration: 2092 --> 0.0
0.0


KeyboardInterrupt: 